# Prepare robots first  
* run panda repeater on controller pc  

```
ssh panda@192.168.0.172
roslaunch panda_ros_repeater joint_velocity_repeater.launch robot_ip:=192.168.0.13 load_gripper:=false

```

* Keep indy connected to conty to bypass conty-connection delay bug

# initialize notebook

In [1]:
from __future__ import print_function
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; } </style>"))
import matplotlib.pyplot as plt

In [2]:
import os
import sys
os.chdir(os.path.dirname(os.getcwd()))

# Initialize constants

In [3]:
from pkg.marker_config import *
from pkg.constraint_graph import *
from pkg.constraint.constraint_action import *
from pkg.constraint.constraint_object import *
from pkg.constants import *
from pkg.utils.plot_utils import *
from pkg.utils.utils import *
from pkg.environment_builder import *
from pkg.ui.ui_broker import *
from pkg.controller.combined_robot import *

gtimer = GlobalTimer.instance()
gtimer.reset()

In [4]:
crob = CombinedRobot(connection_list=(False, False))

connection_list
(False, False)


In [5]:
USE_CAM = False

# initialize graph & ui

In [6]:
if "cam" not in locals():
    
    cam = StereoCamera.instance() if USE_CAM else None

# set urdf
xcustom, JOINT_NAMES, LINK_NAMES, urdf_content = set_custom_robots(crob.robots_on_scene, XYZ_RPY_ROBOTS_DEFAULT, crob.custom_limits, start_rviz=True)
ghnd = GeometryHandle(urdf_content)


graph = ConstraintGraph(ghnd=ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES, 
                        urdf_content=urdf_content, combined_robot=crob)
graph.set_camera(cam)
#graph.set_cam_robot_collision(_add_cam_poles=USE_CAM)
graph.set_rviz()

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(graph)
ui_broker.start_server()

# set rviz
graph.set_rviz(crob.home_pose)
ui_broker.set_tables()

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


# Calibrate camera

In [7]:
if USE_CAM:
    cam.calibrate(ghnd)
    graph.set_cam_robot_collision()
    graph.set_rviz()

# Update Robots

In [8]:
if USE_CAM:
    # btn: detect robots
    crob.detect_robots(cam)

    # btn: apply
    xcustom, JOINT_NAMES, LINK_NAMES, urdf_content = set_custom_robots(crob.robots_on_scene, crob.xyz_rpy_robots, crob.custom_limits, start_rviz=True)
    ghnd = GeometryHandle(urdf_content)


    graph.clear_markers()
    graph.clear_highlight()
    graph.ghnd.clear()
    timer.sleep(1)
    graph.__init__(ghnd=ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES, 
                       urdf_content=urdf_content, combined_robot=crob)
    add_geometry_items(graph.urdf_content, ghnd, color=(0, 1, 0, 0.3), display=True, collision=True,
                       exclude_link=["panda1_link7"])
    graph.set_cam_robot_collision()
    graph.set_rviz()

generate table - Geometry
generate table - Object
generate table - Handle
generate table - Binder
load: 1geos.geometry
button clicked
button clicked
add indy0_link3_Cylinder_0
Please create a subscriber to the marker
add wall
add indy0_link0_Cylinder_0
add panda1_link4_Cylinder_0
add indy0_link5_Cylinder_0
add indy0_link6_Cylinder_0
add panda1_link1_Cylinder_0
add panda1_leftfinger_Cylinder_0
add goal
add floor
add indy0_link4_Cylinder_0
add panda1_link6_Cylinder_1
add panda1_link6_Cylinder_0
add panda1_link6_Cylinder_2
add panda1_link5_Cylinder_0
add panda1_link5_Cylinder_1
add panda1_hand_Cylinder_1
add panda1_hand_Cylinder_0
add indy0_tcp_Cylinder_0
add panda1_link0_Cylinder_1
add indy0_tcp_Cylinder_2
add indy0_tcp_Cylinder_3
add indy0_link2_Cylinder_1
add indy0_link2_Cylinder_0
add panda1_link3_Cylinder_0
add panda1_link0_Cylinder_0
add box3
add panda1_hand_Cylinder_2
add box1
add panda1_rightfinger_Cylinder_0
add box2
add indy0_link1_Cylinder_0
add indy0_tcp_Cylinder_1
add pole_ca

## Load from UI and go on

In [9]:
1

1

# Set Planner

In [10]:
from pkg.planner.moveit.moveit_planner import MoveitPlanner

In [11]:
mplan = MoveitPlanner(joint_names=graph.joint_names, link_names=graph.link_names, urdf_path=graph.urdf_path, urdf_content=graph.urdf_content,
                      robot_names=graph.combined_robot.robot_names, binder_links=[v.object.link_name for v in graph.binder_dict.values()], ghnd=graph.ghnd)
graph.set_planner(mplan)

In [12]:
# hplan = HybridPlanner(joint_names=graph.joint_names, link_names=graph.link_names,
#                       urdf_path=graph.urdf_path, urdf_content=graph.urdf_content,
#                       robot_names=graph.combined_robot.robot_names,
#                       binder_links=[v.object.link_name for v in graph.binder_dict.values()], ghnd=graph.ghnd)
# eplan = etasl_planner(joint_names = graph.joint_names, link_names = graph.link_names, urdf_path = graph.urdf_path)
# graph.set_planner(eplan)

# Set Sampler

In [13]:
from pkg.sampler.object_a_star import *
sampler = ObjectAstarSampler(graph)
graph.set_sampler(sampler)

# Prepare action

In [14]:
sampler.build_graph()

In [15]:
initial_state = State(tuple([(oname,)+ graph.object_dict[oname].binding for oname in graph.object_list]), 
                      {oname: graph.object_dict[oname].object.Toff for oname in graph.object_list}, 
                      graph.get_real_robot_pose(), graph)

In [16]:
dt_sim = 0.04
T_step = 10
N_fullstep = int(T_step / dt_sim)
graph.set_object_state(initial_state)
graph.show_pose(graph.get_real_robot_pose())

goal_nodes_1 = get_goal_nodes(initial_state.node, "box1", "goal_bd")
goal_nodes = []
for gnode in goal_nodes_1:
    goal_nodes += get_goal_nodes(gnode, "box2", "floor")

# Convert dat

In [17]:
 
def get_action_boxes(graph, from_state, to_state, redundancy_dict, mplan=None, L_CELL=0.2, Nwdh=(15,15,15), BASE_LINK='base_link', center_height=0.75):
    if mplan is None:
        mplan = graph.planner
    graph.set_object_state(from_state)
    binding_list = graph.get_slack_bindings(from_state, to_state)
    obj_name, ap_name, binder_name = binding_list[0]
    redundancy = redundancy_dict[obj_name] if redundancy_dict else None
    self, group_name_handle, group_name_binder = mplan, None, None
    binder = self.binder_dict[binder_name]
    obj = self.object_dict[obj_name]
    handle = obj.action_points_dict[ap_name]
    point_add, rpy_add = calc_redundancy(redundancy, binder)
    T_handle = handle.Toff_lh
    T_binder = np.matmul(binder.Toff_lh, SE3(Rot_rpy(rpy_add), point_add))
    if len(self.planner.group_names)==1:
        group_name_handle = self.planner.group_names if handle.object.link_name in self.urdf_content.parent_map else []
        group_name_binder = self.planner.group_names if binder.object.link_name in self.urdf_content.parent_map else []
    else:
        group_name_handle = group_name_handle or [gname for gname in self.planner.group_names if gname in handle.object.link_name]
        group_name_binder = group_name_binder or [gname for gname in self.planner.group_names if gname in binder.object.link_name]

    if group_name_binder and not group_name_handle:
        group_name = group_name_binder[0]
        tool, T_tool = binder, T_binder
        target, T_tar = handle, T_handle
    elif group_name_handle and not group_name_binder:
        group_name = group_name_handle[0]
        tool, T_tool = handle, T_handle
        target, T_tar = binder, T_binder
    else:
        print(binder.name, obj.object.name)
        print(group_name_binder, group_name_handle)
        print("uncontrollable binding")
        raise(RuntimeError("uncontrollable binding"))
    #     return [], [0]*self.planner.joint_num, None, False

    T_tar_tool = np.matmul(T_tar, SE3_inv(T_tool))
    goal_pose = tuple(T_tar_tool[:3,3]) \
                +tuple(Rotation.from_dcm(T_tar_tool[:3,:3]).as_quat())

    CENTER_OFFSET = tuple(np.multiply(Nwdh[:2], L_CELL) / 2)+(center_height,)
    T_ho = SE3_inv(target.Toff_oh)
    T_tar_o = np.matmul(get_tf(target.object.link_name, Q_dict, self.urdf_content), np.matmul(T_tar, T_ho))
    T_tool_o = np.matmul(get_tf(tool.object.link_name, Q_dict, self.urdf_content), np.matmul(T_tool, T_ho))

    T_tar_o[:3,3]+= CENTER_OFFSET
    T_tool_o[:3,3]+= CENTER_OFFSET

    cell_init, verts_init = get_cell_verts(GEOTYPE.BOX, T_tool_o, obj.object.dims, L_CELL, Nwdh)
    chain_init = chain_dict[tool.object.link_name]
    cell_goal, verts_goal = get_cell_verts(GEOTYPE.BOX, T_tar_o, obj.object.dims, L_CELL, Nwdh)
    chain_goal = chain_dict[target.object.link_name]
    return cell_init, verts_init, chain_init, cell_goal, verts_goal, chain_goal

In [17]:
import numpy as np
import SharedArray as sa

# Attach it as a different array. This can be done from another
# python interpreter as long as it runs on the same computer.
scene_base_data_p = sa.attach("shm://scene_base_data")
cell_init_p = sa.attach("shm://cell_init")
verts_init_p = sa.attach("shm://verts_init")
chain_init_p = sa.attach("shm://chain_init")
cell_goal_p = sa.attach("shm://cell_goal")
verts_goal_p = sa.attach("shm://verts_goal")
chain_goal_p = sa.attach("shm://chain_goal")

In [31]:
for gtem in ghnd:
    if gtem.name in ["wall", "floor", "pole_cam0", "pole_cam1"]:
        gtem.collision = False
        gtem.display = False
    

In [19]:
gtimer.reset()
gtimer.tic("convert_workspace")
ghnd_new= graph.convert_workspace(ghnd)
gtimer.toc("convert_workspace")

ignore box3 out of workspace


12.598991394042969

In [33]:
graph.replace_ghnd(ghnd_new)

In [34]:
graph.set_rviz()

Exception KeyboardInterrupt in <bound method GeoMarker.__del__ of <pkg.geometry.ros_rviz.GeoMarker instance at 0x7f6b9585b410>> ignored


In [22]:
from_state = initial_state
gtimer.tic("convert_scene")
scene_base_data, ctem_names, ctem_cells, chain_dict = graph.convert_scene(ghnd_new, state=from_state, only_base=False)
gtimer.toc("convert_scene")

8.029937744140625

In [23]:
gtimer.tic("copy_scene_base_data")
np.copyto(scene_base_data_p, scene_base_data)
gtimer.toc("copy_scene_base_data")

1.898050308227539

In [24]:
print(gtimer)

convert_workspace: 	13.0 ms/1 = 12.599 ms (12.599/12.599)
convert_scene: 	8.0 ms/1 = 8.03 ms (8.03/8.03)
test_links: 	1.0 ms/1 = 0.755 ms (0.755/0.755)
T_chain: 	1.0 ms/1 = 0.656 ms (0.656/0.656)
get_chain: 	0.0 ms/21 = 0.003 ms (0.001/0.012)
test_joints: 	0.0 ms/1 = 0.101 ms (0.101/0.101)
calc_cell: 	3.0 ms/1 = 2.579 ms (2.579/2.579)
list_ctems: 	1.0 ms/1 = 0.734 ms (0.734/0.734)
calc_ctems: 	2.0 ms/1 = 1.536 ms (1.536/1.536)
rearrange_cell_array: 	1.0 ms/4 = 0.208 ms (0.185/0.239)
copy_scene_base_data: 	2.0 ms/1 = 1.898 ms (1.898/1.898)



In [25]:
state = initial_state
leaf = sampler.node_dict[initial_state.onode][0]
Q_dict = list2dict(state.Q, graph.joint_names)
available_binding_dict = sampler.get_available_binding_dict(state, leaf, Q_dict)

In [26]:
gtimer.reset()
BATCH_SIZE = 12
cell_init_b, verts_init_b, chain_init_b, cell_goal_b, verts_goal_b, chain_goal_b = [], [], [], [], [], []
action_batch = []
for _ in range(BATCH_SIZE):
    gtimer.tic("get_action_boxes")
    to_state, redundancy_dict = sampler.sample_leaf_state(state, available_binding_dict, leaf)
    action_batch.append((to_state, redundancy_dict))
    cell_init, verts_init, chain_init, cell_goal, verts_goal, chain_goal = get_action_boxes(
        graph, from_state, to_state, redundancy_dict, mplan=mplan, L_CELL=0.2, Nwdh=(15,15,15), BASE_LINK='base_link')
    cell_init_b.append(cell_init)
    verts_init_b.append(verts_init)
    chain_init_b.append(chain_init)
    cell_goal_b.append(cell_goal)
    verts_goal_b.append(verts_goal)
    chain_goal_b.append(chain_goal)
    gtimer.toc("get_action_boxes")
cell_init_b = np.array(cell_init_b)
verts_init_b = np.array(verts_init_b)
chain_init_b = np.array(chain_init_b)
cell_goal_b = np.array(cell_goal_b)
verts_goal_b = np.array(verts_goal_b)
chain_goal_b = np.array(chain_goal_b)

In [27]:
np.copyto(cell_init_p, cell_init_b)
np.copyto(verts_init_p, verts_init_b)
np.copyto(chain_init_p, chain_init_b)
np.copyto(cell_goal_p, cell_goal_b)
np.copyto(verts_goal_p, verts_goal_b)
np.copyto(chain_goal_p, chain_goal_b)

# test

In [28]:
WDH_MIN_RBT = (0.75, 0.75, 0.5)
WDH_MAX_RBT = (2.25, 2.25, 1.0)
H_MEAN = np.mean([WDH_MAX_RBT[2], WDH_MIN_RBT[2]])
Nwdh = (15,15,15)
L_CELL= 0.2

In [29]:
joint_num = crob.joint_num

N_vtx_box = 3*8
N_mask_box = 1
N_joint_box = joint_num
N_label_box = N_vtx_box+N_mask_box+N_joint_box
N_vtx_cyl = 3*2+1
N_mask_cyl = 1
N_joint_cyl = joint_num
N_label_cyl = N_vtx_cyl+N_mask_cyl+N_joint_cyl
N_vtx_init = 3*8
N_mask_init = 1
N_joint_init = joint_num
N_label_init = N_vtx_init+N_mask_init+N_joint_init
N_vtx_goal = 3*8
N_mask_goal = 1
N_joint_goal = joint_num
N_label_goal = N_vtx_goal+N_mask_goal+N_joint_goal
N_joint_label = 9*joint_num
N_cell_label = N_label_box+N_label_cyl+N_label_init+N_label_goal + N_joint_label
N_BEGIN_CYL = N_vtx_box+N_mask_box+N_joint_box
N_BEGIN_INIT = N_BEGIN_CYL+N_vtx_cyl+N_mask_cyl+N_joint_cyl
N_BEGIN_GOAL = N_BEGIN_INIT+N_vtx_init+N_mask_init+N_joint_init

In [30]:
succ_batch = []
for to_state, redundancy_dict in action_batch:
    traj, new_state, error, succ = graph.test_transition(from_state, to_state, display=False,
                                                         redundancy_dict=redundancy_dict,
                                                         dt_vis=0.01,
                                                         print_expression=False)
    succ_batch.append(succ)
print(succ_batch)

[False, True, True, True, True, True, False, True, True, False, True, False]


In [51]:
scene_base_batch = np.stack([scene_base_data_p]*BATCH_SIZE)

In [52]:
for i_b, cell, verts, chain in zip(range(BATCH_SIZE), cell_init_p, verts_init_p, chain_init_p):
    scene_base_batch[i_b, cell[0], cell[1], cell[2], 
                     N_BEGIN_INIT:
                     N_BEGIN_INIT+N_vtx_init] = verts
    scene_base_batch[i_b, cell[0], cell[1], cell[2], 
                     N_BEGIN_INIT+N_vtx_init:
                     N_BEGIN_INIT+N_vtx_init+N_mask_init] = 1
    scene_base_batch[i_b, cell[0], cell[1], cell[2], 
                     N_BEGIN_INIT+N_vtx_init+N_mask_init:
                     N_BEGIN_INIT+N_vtx_init+N_mask_init+N_joint_init] = chain

In [53]:
for i_b, cell, verts, chain in zip(range(BATCH_SIZE), cell_goal_p, verts_goal_p, chain_goal_p):
    scene_base_batch[i_b, cell[0], cell[1], cell[2], 
                     N_BEGIN_GOAL:
                     N_BEGIN_GOAL+N_vtx_goal] = verts
    scene_base_batch[i_b, cell[0], cell[1], cell[2], 
                     N_BEGIN_GOAL+N_vtx_goal:
                     N_BEGIN_GOAL+N_vtx_goal+N_mask_goal] = 1
    scene_base_batch[i_b, cell[0], cell[1], cell[2], 
                     N_BEGIN_GOAL+N_vtx_goal+N_mask_goal:
                     N_BEGIN_GOAL+N_vtx_goal+N_mask_goal+N_joint_goal] = chain

In [54]:
scene_data = scene_base_batch[0]

centers = get_centers(Nwdh, L_CELL)-(1.5,1.5,H_MEAN)
#centers = get_centers(Nwdh, L_CELL)
for idx, center, cell_dat in zip(range(Nwdh[0]*Nwdh[1]*Nwdh[2]), centers.reshape((-1,3)), scene_data.reshape((-1, scene_data.shape[-1]))):
    gtem, mask, chain = get_box_diplay(ghnd, "box_{}".format(idx), cell_dat, 0, joint_num, center)
    if mask:
        graph.add_marker(gtem)
    gtem, mask, chain = get_cyl_diplay(ghnd, "cyl_{}".format(idx), cell_dat, N_BEGIN_CYL, joint_num, center)
    if mask:
        graph.add_marker(gtem)
    gtem, mask, chain = get_box_diplay(ghnd, "init_{}".format(idx), cell_dat, N_BEGIN_INIT, joint_num, center, color=(0,0,0,0.8))
    if mask:
        graph.add_marker(gtem)
    gtem, mask, chain = get_box_diplay(ghnd, "goal_{}".format(idx), cell_dat, N_BEGIN_GOAL, joint_num, center, color=(0.8,0.8,0.8,0.8))
    if mask:
        graph.add_marker(gtem)

In [57]:

for idx, center, cell_dat in zip(range(Nwdh[0]*Nwdh[1]*Nwdh[2]), centers.reshape((-1,3)), scene_data.reshape((-1, scene_data.shape[-1]))):
    gtem, mask, chain = get_box_diplay(ghnd, "box_{}".format(idx), cell_dat, 0, joint_num, center)
    if mask:
        graph.remove_geometry(gtem)
    gtem, mask, chain = get_cyl_diplay(ghnd, "cyl_{}".format(idx), cell_dat, N_BEGIN_CYL, joint_num, center)
    if mask:
        graph.remove_geometry(gtem)
    gtem, mask, chain = get_box_diplay(ghnd, "init_{}".format(idx), cell_dat, N_BEGIN_INIT, joint_num, center, color=(0,0,0,0.8))
    if mask:
        graph.remove_geometry(gtem)
    gtem, mask, chain = get_box_diplay(ghnd, "goal_{}".format(idx), cell_dat, N_BEGIN_GOAL, joint_num, center, color=(0,0,0,0.8))
    if mask:
        graph.remove_geometry(gtem)

In [39]:
graph.set_rviz()

In [55]:
graph.show_motion(traj)

In [34]:
print(gtimer)

get_action_boxes: 	1.0 ms/1 = 0.927 ms (0.927/0.927)



In [35]:
scene_base_data.shape

(15, 15, 15, 252)

## visualization testing verts

In [31]:
cell, verts = cell_goal, verts_goal
print(cell, verts)
target_center = np.multiply(cell, L_CELL)+0.5*L_CELL-CENTER_OFFSET
target_cell = ghnd.create_safe(name="target_cell", gtype=GEOTYPE.BOX, link_name=BASE_LINK,
                               dims=(L_CELL, L_CELL, L_CELL), center=target_center, rpy=(0, 0, 0), collision=False, display=True,
                               color=(1,0,0,0.3))
graph.add_marker(target_cell)
verts_res = np.reshape(verts, (-1,3))
target_verts = []
for i_v, vert in enumerate(verts_res):
    target_vert = ghnd.create_safe(name="target_vert_%d"%i_v, gtype=GEOTYPE.SPHERE, link_name=BASE_LINK,
                                   dims=(0.01, 0.01, 0.01), center=target_center+vert, rpy=(0, 0, 0), collision=False, display=True,
                                   color=(1,0,0,0.7))
    target_verts.append(target_vert)
    graph.add_marker(target_vert)

cell, verts = cell_init, verts_init
print(cell, verts)
tool_center = np.multiply(cell, L_CELL)+0.5*L_CELL-CENTER_OFFSET
tool_cell = ghnd.create_safe(name="tool_cell", gtype=GEOTYPE.BOX, link_name=BASE_LINK,
                               dims=(L_CELL, L_CELL, L_CELL), center=tool_center, rpy=(0, 0, 0), collision=False, display=True,
                               color=(1,0,0,0.3))
graph.add_marker(tool_cell)
verts_res = np.reshape(verts, (-1,3))
T_tool_inv = SE3_inv(get_tf(tool.object.link_name, Q_dict, mplan.urdf_content))
tool_verts = []
for i_v, vert in enumerate(verts_res):
    T_vert = SE3(np.identity(3), tool_center+vert)
    T_vert_bind = np.matmul(T_tool_inv, T_vert)
    tool_vert = ghnd.create_safe(name="tool_vert_%d"%i_v, gtype=GEOTYPE.SPHERE, link_name=BASE_LINK,
                                   dims=(0.01, 0.01, 0.01), center=T_vert[:3,3], rpy=(0, 0, 0), collision=False, display=True,
                                   color=(1,0,0,0.7))
    tool_vert_bind = ghnd.create_safe(name="tool_vert_bind_%d"%i_v, gtype=GEOTYPE.SPHERE, link_name=tool.object.link_name,
                                   dims=(0.01, 0.01, 0.01), center=T_vert_bind[:3,3], rpy=(0, 0, 0), collision=False, display=True,
                                   color=(0,0,1,0.7))
    tool_verts.append(tool_vert)
    tool_verts.append(tool_vert_bind)
    graph.add_marker(tool_vert)
    graph.add_marker(tool_vert_bind)

[6 6 7] [-0.0537812   0.08276885  0.05602929 -0.03396299  0.03686523  0.05571912
 -0.05413367  0.08295453  0.00603088 -0.03431546  0.03705091  0.00572071
 -0.09968452  0.06294908  0.05627929 -0.07986631  0.01704546  0.05596912
 -0.10003699  0.06313476  0.00628088 -0.08021878  0.01723114  0.0059707 ]
[4 6 9] [-0.01232297  0.13403402 -0.00951462 -0.06231839  0.13471089 -0.00952867
 -0.01285815  0.09515673  0.02192214 -0.06285357  0.0958336   0.02190809
 -0.01273762  0.10259999 -0.04839559 -0.06273304  0.10327686 -0.04840964
 -0.0132728   0.0637227  -0.01695883 -0.06326822  0.06439957 -0.01697288]


In [32]:
graph.show_motion(traj)

In [33]:
graph.remove_geometry(target_cell)
graph.remove_geometry(tool_cell)
for vert in target_verts:
    graph.remove_geometry(vert)
for vert in tool_verts:
    graph.remove_geometry(vert)

In [34]:
graph.show_pose(crob.home_pose)

## search

In [17]:
sampler.search_graph(
    initial_state = initial_state, goal_nodes = goal_nodes,
    tree_margin = 4, depth_margin = 4, 
    terminate_on_first = True, N_search = 100, multiprocess=False,
    display=False, dt_vis=dt_sim/4, verbose = True, print_expression=False, error_skip=0, traj_count=DEFAULT_TRAJ_COUNT,
    **dict(N=N_fullstep, dt=dt_sim, vel_conv=0.5e-2, err_conv=1e-3))

node: (u'floor', u'floor')->(u'grip1', u'floor') = success
Goal cost:2->1 / Init cost:0->1 / branching: 0->1 (0.13/600.0 s, steps/err: 20(121.315956116 ms)/2.19503012849)
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node

node: (u'grip0', u'floor')->(u'goal', u'floor') = fail
node: (u'grip0', u'floor')->(u'goal', u'floor') = fail
node: (u'grip0', u'floor')->(u'goal', u'floor') = fail
node: (u'floor', u'floor')->(u'grip0', u'floor') = fail
node: (u'floor', u'floor')->(u'grip1', u'floor') = fail
node: (u'floor', u'floor')->(u'grip1', u'floor') = fail
node: (u'floor', u'floor')->(u'grip1', u'floor') = success
Goal cost:2->1 / Init cost:0->1 / branching: 0->5 (3.76/600.0 s, steps/err: 15(46.8330383301 ms)/1.59083164108)
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
n

node: (u'floor', u'floor')->(u'grip1', u'floor') = fail
node: (u'floor', u'floor')->(u'grip1', u'floor') = fail
node: (u'floor', u'floor')->(u'grip1', u'floor') = fail
node: (u'floor', u'floor')->(u'grip0', u'floor') = fail
node: (u'floor', u'floor')->(u'grip0', u'floor') = fail
node: (u'floor', u'floor')->(u'grip1', u'floor') = fail
node: (u'floor', u'floor')->(u'grip1', u'floor') = fail
node: (u'floor', u'floor')->(u'grip1', u'floor') = success
Goal cost:2->1 / Init cost:0->1 / branching: 0->9 (8.04/600.0 s, steps/err: 31(150.738954544 ms)/2.41709503499)
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fail
node: (u'grip1', u'floor')->(u'goal', u'floor') = fa

node: (u'grip0', u'floor')->(u'goal', u'floor') = fail
node: (u'grip0', u'floor')->(u'goal', u'floor') = fail
node: (u'grip0', u'floor')->(u'goal', u'floor') = fail
node: (u'grip0', u'floor')->(u'goal', u'floor') = fail
node: (u'grip0', u'floor')->(u'goal', u'floor') = fail
node: (u'grip0', u'floor')->(u'goal', u'floor') = fail
node: (u'grip0', u'floor')->(u'goal', u'floor') = fail
node: (u'grip0', u'floor')->(u'goal', u'floor') = fail
node: (u'grip0', u'floor')->(u'goal', u'floor') = fail
node: (u'grip0', u'floor')->(u'goal', u'floor') = fail
node: (u'grip0', u'floor')->(u'goal', u'floor') = fail
node: (u'grip0', u'floor')->(u'goal', u'floor') = fail
node: (u'grip0', u'floor')->(u'goal', u'floor') = fail
node: (u'grip0', u'floor')->(u'goal', u'floor') = fail
node: (u'floor', u'floor')->(u'grip0', u'floor') = fail
node: (u'floor', u'floor')->(u'grip0', u'floor') = success
Goal cost:2->1 / Init cost:0->1 / branching: 0->13 (11.72/600.0 s, steps/err: 42(56.370973587 ms)/1.9457178655)
nod

(u'grip1', u'box1')
(['panda1'], ['indy0'])
uncontrollable binding
node: (u'grip0', u'floor')->(u'grip1', u'floor') = fail
(u'grip1', u'box1')
(['panda1'], ['indy0'])
uncontrollable binding
node: (u'grip0', u'floor')->(u'grip1', u'floor') = fail
(u'grip1', u'box1')
(['panda1'], ['indy0'])
uncontrollable binding
node: (u'grip0', u'floor')->(u'grip1', u'floor') = fail
(u'grip1', u'box1')
(['panda1'], ['indy0'])
uncontrollable binding
node: (u'grip0', u'floor')->(u'grip1', u'floor') = fail
(u'grip1', u'box1')
(['panda1'], ['indy0'])
uncontrollable binding
node: (u'grip0', u'floor')->(u'grip1', u'floor') = fail
(u'grip1', u'box1')
(['panda1'], ['indy0'])
uncontrollable binding
node: (u'grip0', u'floor')->(u'grip1', u'floor') = fail
(u'grip1', u'box1')
(['panda1'], ['indy0'])
uncontrollable binding
node: (u'grip0', u'floor')->(u'grip1', u'floor') = fail
(u'grip1', u'box1')
(['panda1'], ['indy0'])
uncontrollable binding
node: (u'grip0', u'floor')->(u'grip1', u'floor') = fail
(u'grip1', u'box

...

node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
n

node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'floor', u'grip0')->(u'grip1', u'grip0') = fail
node: (u'floor', u'grip0')->(u'grip1', u'grip0') = fail
node: (u'floor', u'grip0')->(u'grip1', u'grip0') = fail
node: (u'floor', u'grip0')->(u'grip1', u'grip0') = success
Goal cost:3->2 / Init cost:1->2 / branching: 88->92 (114.66/600.0 s, steps/err: 24(118.340969086 ms)/2.09446995

node: (u'floor', u'grip0')->(u'grip1', u'grip0') = fail
node: (u'floor', u'grip0')->(u'grip1', u'grip0') = fail
node: (u'floor', u'grip0')->(u'grip1', u'grip0') = fail
node: (u'floor', u'grip0')->(u'grip1', u'grip0') = fail
node: (u'floor', u'grip0')->(u'grip1', u'grip0') = fail
node: (u'floor', u'grip0')->(u'grip1', u'grip0') = success
Goal cost:3->2 / Init cost:1->2 / branching: 88->94 (117.61/600.0 s, steps/err: 25(125.274896622 ms)/1.9047329338)
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'f

node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'grip1', u'floor') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
n

node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'grip1', u'grip0')->(u'goal', u'grip0') = fail
node: (u'floor', u'grip0')->(u'grip1', u'grip0') = fail
node: (u'floor', u'grip0')->(u'grip1', u'grip0') = fail
node: (u'floor', u'grip0')->(u'grip1', u'grip0') = success
Goal cost:3->2 / Init cost:1->2 / branching: 97->99 (124.8/600.0 s, steps/err: 31(153.210878372 ms)/1.5690625139)


# Detect environment

In [9]:
env_gen_dict, objectPose_dict, corner_dict, color_image, rs_objectPose_dict, rs_corner_dict, rs_image = cam.detect_environment(ghnd)
add_objects_gen(graph, env_gen_dict)
graph.set_rviz()

# Reset robot connection

In [10]:
graph.combined_robot.reset_connection([False, False])

connection_list
[False, False]


# Go home

In [11]:
if all(graph.combined_robot.connection_list):
    graph.combined_robot.joint_make_sure(graph.combined_robot.home_pose)

## Register binders

In [12]:
graph.register_binder(name='grip1', _type=Gripper2Tool, point=[0,0,0.112], rpy=[-np.pi/2,0,0], link_name="panda1_hand")
graph.register_binder(name='grip0', _type=Gripper2Tool, point=[0,0,0.14], rpy=[-np.pi/2,0,0], link_name="indy0_tcp")
graph.register_binder(name='floor', _type=PlacePlane)

## detect movable

In [13]:
BINDER_DICT = {'goal_bd': dict(_type=PlacePlane, object_name="goal", point=[0,0,0.02], rpy=[0,0,0])}
OBJECT_DICT = {'box1': dict(_type=BoxAction, hexahedral=True),
               'box2': dict(_type=BoxAction, hexahedral=True)}

In [14]:
objectPose_dict_mv, corner_dict_mv, color_image, aruco_map_mv = detect_objects(graph.cam.aruco_map, graph.cam.dictionary, graph.cam.ref_tuple)
put_point_dict = graph.register_object_gen(objectPose_dict_mv, BINDER_DICT, OBJECT_DICT, link_name="base_link")

In [15]:
graph.set_rviz()

button clicked
button clicked
button action done
button action done
('saved on :', u'1geos.geometry')
generate table - Geometry
generate table - Handle
generate table - Object
generate table - Binder
filname is empty
generate table - Handle
generate table - Binder
generate table - Geometry
generate table - Object
button clicked
button clicked
button action done
button action done
('saved on :', u'1geos.geometry')
button clicked
button clicked
button action done
button action done
('saved on :', u'2.binder.binder')
button clicked
button clicked
button action done
button action done
('saved on :', u'3objects.object')
load: 1geos.geometry
button clicked
button clicked
button action done
button action done
generate table - Geometry
generate table - Object
generate table - Handle
generate table - Binder
filname is empty
filname is empty
filname is empty
load: 2.binder.binder
button clicked
button clicked
button action done
button action done
generate table - Geometry
generate table - Object

In [39]:
1

1

button clicked
button clicked
button action done
button action done
('saved on :', u'objects.object')
load: objects.object
button clicked
button clicked
button action done
button action done
generate table - Geometry
generate table - Object
generate table - Handle
generate table - Binder
filname is empty
button clicked
button clicked
button action done
button action done
('saved on :', u'binder.binder')


In [16]:
graph.show_pose(graph.get_real_robot_pose())

## Register object binders

In [17]:
register_hexahedral_binder(graph, object_name='box1', _type=PlacePlane)
register_hexahedral_binder(graph, object_name='box2', _type=PlacePlane)

# Set Planner

In [18]:
from pkg.planner.moveit.moveit_planner import MoveitPlanner

In [19]:
hplan = HybridPlanner(joint_names=graph.joint_names, link_names=graph.link_names,
                      urdf_path=graph.urdf_path, urdf_content=graph.urdf_content,
                      robot_names=graph.combined_robot.robot_names,
                      binder_links=[v.object.link_name for v in graph.binder_dict.values()], ghnd=graph.ghnd)
graph.set_planner(hplan)

In [20]:
# eplan = etasl_planner(joint_names = graph.joint_names, link_names = graph.link_names, urdf_path = graph.urdf_path)
# graph.set_planner(eplan)

# Set Sampler

In [21]:
from pkg.sampler.object_a_star import *
sampler = ObjectAstarSampler(graph)
graph.set_sampler(sampler)

# Search Graph

In [ ]:
gtimer.reset()

In [ ]:
sampler.build_graph()
print(graph.gtimer)

In [ ]:
OBJECT_DICT = {k:dict(_type=v.__class__) for k,v in graph.object_dict.items()}
objectPose_dict_mv, corner_dict_mv, color_image, aruco_map_mv = \
    detect_objects(graph.cam.aruco_map, graph.cam.dictionary)
xyz_rvec_mv_dict, put_point_dict, up_point_dict = calc_put_point(ghnd, 
    objectPose_dict_mv, graph.cam.aruco_map, OBJECT_DICT, graph.cam.ref_tuple)
update_geometries(ghnd, objectPose_dict_mv.keys(), objectPose_dict_mv, graph.cam.ref_tuple[1])
graph.set_rviz()
dt_sim = 0.04
T_step = 10
N_fullstep = int(T_step / dt_sim)
initial_state = State(tuple([(oname, put_point_dict[oname],'floor') for oname in graph.object_list]), 
                      {oname: graph.object_dict[oname].object.Toff for oname in graph.object_list}, 
                      graph.get_real_robot_pose(), graph)
binding_dict = match_point_binder(graph, initial_state, objectPose_dict_mv)
initial_state = State(tuple([(oname, put_point_dict[oname],binding_dict[oname]) for oname in graph.object_list]), 
                      {oname: graph.object_dict[oname].object.Toff for oname in graph.object_list}, 
                      graph.get_real_robot_pose(), graph)
graph.set_object_state(initial_state)
graph.show_pose(graph.get_real_robot_pose())

goal_nodes_1 = get_goal_nodes(initial_state.node, "box1", "goal_bd")
goal_nodes = []
for gnode in goal_nodes_1:
    goal_nodes += get_goal_nodes(gnode, "box2", "floor")
# goal_nodes = goal_nodes[3:4]

sampler.search_graph(
    initial_state = initial_state, goal_nodes = goal_nodes,
    tree_margin = 4, depth_margin = 4, 
    terminate_on_first = True, N_search = 100, multiprocess=True,
    display=True, dt_vis=dt_sim/4, verbose = True, print_expression=False, error_skip=0, traj_count=DEFAULT_TRAJ_COUNT,
    **dict(N=N_fullstep, dt=dt_sim, vel_conv=0.5e-2, err_conv=1e-3))

schedule_dict = sampler.find_schedules()
schedule_sorted = sampler.sort_schedule(schedule_dict)

In [ ]:
print(gtimer)
sampler.quiver_snodes()

# Replay schedule

In [ ]:
schedule_dict = sampler.find_schedules()
schedule_sorted = sampler.sort_schedule(schedule_dict)
N_fullstep = 500
dt_sim = 0.04
# dt_sim = 0.04
for schedule in schedule_sorted:
    print(schedule)
for schedule, i_s in zip(schedule_sorted[:2], range(len(schedule_sorted))):
    graph.set_object_state(sampler.snode_dict[0].state)
    graph.show_pose(sampler.snode_dict[0].state.Q)
    timer.sleep(0.1)
#     try:
    e = graph.replay(schedule, N=N_fullstep, dt=dt_sim, 
                     vel_conv=0.5e-2, err_conv=1e-3, error_skip=0, dt_vis=0.02)
#     except Exception as e:
#         print(e)

# Simulate traj online

In [ ]:
eplan = etasl_planner(ghnd=ghnd, joint_names = graph.joint_names, link_names = graph.link_names, urdf_path = graph.urdf_path)

In [ ]:
snode_schedule = graph.sampler.find_best_schedule(schedule_sorted)
# snode_schedule = graph.sampler.idxSchedule2SnodeScedule(schedule_sorted[0], graph.combined_robot.home_pose)

eplan.update(graph)

with DynamicDetector(eplan.online_names, graph.cam.aruco_map, graph.cam.dictionary, graph.cam.rs_config, graph.cam.T_c12, graph.cam.ref_tuple[1]) as dynamic_detector, \
        RvizPublisher(graph, eplan.online_names) as rviz_pub:
    e_sim = graph.execute_schedule_online(snode_schedule, eplan, control_freq=DEFAULT_TRAJ_FREQUENCY, playback_rate=0.5,
                                  vel_conv=0, err_conv=1e-3, T_step = 100, on_rviz=True, 
                                  dynamic_detector=dynamic_detector, rviz_pub=rviz_pub, obs_K="40")

# Execute traj online

In [ ]:
graph.combined_robot.reset_connection([True, True])

In [ ]:
snode_schedule = graph.sampler.find_best_schedule(schedule_sorted)
eplan.update(graph)

with DynamicDetector(eplan.online_names, graph.cam.aruco_map, graph.cam.dictionary, graph.cam.rs_config, graph.cam.T_c12, graph.cam.ref_tuple[1]) as dynamic_detector, \
        RvizPublisher(graph, eplan.online_names) as rviz_pub:
    e_sim = graph.execute_schedule_online(snode_schedule, eplan, control_freq=DEFAULT_TRAJ_FREQUENCY, playback_rate=0.2,
                                  vel_conv=0, err_conv=1e-3, T_step = 100, on_rviz=False, 
                                  dynamic_detector=dynamic_detector, rviz_pub=rviz_pub, obs_K="40")

In [ ]:
print(gtimer)

In [ ]:
plt.plot(gtimer.timelist_dict["move_wait"])
plt.axis([0,15000,0,50])

In [ ]:
plt.plot(gtimer.timelist_dict["send-robot-0"])
plt.axis([0,6000,0,10])

In [ ]:
plt.plot(gtimer.timelist_dict["count-robot-0"])
# plt.axis([0,6000,0,10])

In [ ]:
plt.plot(gtimer.timelist_dict["send-robot-1"])
plt.axis([0,6000,0,10])

In [ ]:
plt.plot(gtimer.timelist_dict["count-robot-1"])
# plt.axis([0,6000,0,20])

# onestep

In [ ]:
loop_process = graph
loop_process.stop_now = False
while not loop_process.stop_now:
    print("wait for button input")
    graph.indy.connect_and(graph.indy.wait_di, 16)
    if loop_process.stop_now:
        break
    kn_config, rs_config, T_c12 = calibrate_stereo(aruco_map, dictionary)
    objectPose_dict_mv, corner_dict_mv, color_image, aruco_map_mv = \
        detect_objects(MOVABLE_GENERATORS, aruco_map, dictionary)
    objectPose_dict_mv.update({'floor': objectPose_dict['floor']})
    xyz_rvec_mv_dict, put_point_dict, up_point_dict = calc_put_point(
        objectPose_dict_mv, MOVABLE_GENERATORS, OBJECT_DICT, ("floor", objectPose_dict["floor"]))
    update_geometries(MOVABLE_GENERATORS.keys(), objectPose_dict_mv)
    graph.show_pose(ZERO_JOINT_POSE)

    dt_sim = 0.04
    T_step = 10
    N_fullstep = int(T_step / dt_sim)
    gtimer.reset()
    initial_state = State(tuple([(oname, put_point_dict[oname],'floor') for oname in graph.object_list]), 
                          {oname: graph.object_dict[oname].object.get_offset_tf() for oname in graph.object_list}, 
                          (graph.get_real_robot_pose() if graph.connect_indy and graph.connect_panda 
                           else ZERO_JOINT_POSE))
    binding_dict = match_point_binder(graph, initial_state, objectPose_dict_mv)
    initial_state = State(tuple([(oname, put_point_dict[oname],binding_dict[oname]) for oname in graph.object_list]), 
                          {oname: graph.object_dict[oname].object.get_offset_tf() for oname in graph.object_list}, 
                          (graph.get_real_robot_pose() if graph.connect_indy and graph.connect_panda 
                           else ZERO_JOINT_POSE))
    graph.set_object_state(initial_state)
    graph.show_pose(ZERO_JOINT_POSE)

    goal_nodes_1 = get_goal_nodes(initial_state.node, "box1", "goal_bd")
    goal_nodes = []
    for gnode in goal_nodes_1:
        goal_nodes += get_goal_nodes(gnode, "box2", "floor")
    # goal_nodes = goal_nodes[3:4]

    graph.search_graph_mp(
        initial_state = initial_state, goal_nodes = goal_nodes, swept_volume_test_jmotion=False,
        tree_margin = 2, depth_margin = 2, joint_motion_num=10, 
        terminate_on_first = True, N_search = 100, N_loop=1000,
        display=False, dt_vis=dt_sim/40, verbose = True, print_expression=False, error_skip=0,
        **dict(N=N_fullstep, dt=dt_sim, vel_conv=1e-3, err_conv=1e-3, N_step=N_fullstep))

    schedule_dict = graph.find_schedules()
    schedule_sorted = graph.sort_schedule(schedule_dict)
    print(gtimer)
    schedule_dict = graph.find_schedules()
    schedule_sorted = graph.sort_schedule(schedule_dict)

    schedule = schedule_sorted[0]
    state_schedule = graph.idxSchedule2stateScedule(schedule, ZERO_JOINT_POSE)
    obs_names = ["box3"]

    with DynamicDetector(obs_names, aruco_map, dictionary, rs_config, T_c12, objectPose_dict['floor']) as dynamic_detector, \
            RvizPublisher(graph, obs_names) as rviz_pub:
        graph.execute_schedule_online(state_schedule, control_freq=DEFAULT_TRAJ_FREQUENCY, playback_rate=0.5,
                                      vel_conv=1e-2, err_conv=1e-3, T_step = 100, on_rviz=False, 
                                      obs_names=obs_names, dynamic_detector=dynamic_detector, rviz_pub=rviz_pub)
print("===========================================")
print("=================KILLED====================")
print("===========================================")

In [ ]:
def loop_process():
    loop_process.stop_now = False
    while not loop_process.stop_now:
        print("wait for button input")
        graph.indy.connect_and(graph.indy.wait_di, 16)
        if loop_process.stop_now:
            break
        kn_config, rs_config, T_c12 = calibrate_stereo(aruco_map, dictionary)
        objectPose_dict_mv, corner_dict_mv, color_image, aruco_map_mv = \
            detect_objects(MOVABLE_GENERATORS, aruco_map, dictionary)
        objectPose_dict_mv.update({'floor': objectPose_dict['floor']})
        xyz_rvec_mv_dict, put_point_dict, up_point_dict = calc_put_point(
            objectPose_dict_mv, MOVABLE_GENERATORS, OBJECT_DICT, ("floor", objectPose_dict["floor"]))
        update_geometries(MOVABLE_GENERATORS.keys(), objectPose_dict_mv)
        graph.show_pose(ZERO_JOINT_POSE)
        
        dt_sim = 0.04
        T_step = 10
        N_fullstep = int(T_step / dt_sim)
        gtimer.reset()
        initial_state = State(tuple([(oname, put_point_dict[oname],'floor') for oname in graph.object_list]), 
                              {oname: graph.object_dict[oname].object.get_offset_tf() for oname in graph.object_list}, 
                              (graph.get_real_robot_pose() if graph.connect_indy and graph.connect_panda 
                               else ZERO_JOINT_POSE))
        binding_dict = match_point_binder(graph, initial_state, objectPose_dict_mv)
        initial_state = State(tuple([(oname, put_point_dict[oname],binding_dict[oname]) for oname in graph.object_list]), 
                              {oname: graph.object_dict[oname].object.get_offset_tf() for oname in graph.object_list}, 
                              (graph.get_real_robot_pose() if graph.connect_indy and graph.connect_panda 
                               else ZERO_JOINT_POSE))
        graph.set_object_state(initial_state)
        graph.show_pose(ZERO_JOINT_POSE)

        goal_nodes_1 = get_goal_nodes(initial_state.node, "box1", "goal_bd")
        goal_nodes = []
        for gnode in goal_nodes_1:
            goal_nodes += get_goal_nodes(gnode, "box2", "floor")
        # goal_nodes = goal_nodes[3:4]

        graph.search_graph_mp(
            initial_state = initial_state, goal_nodes = goal_nodes, swept_volume_test_jmotion=False,
            tree_margin = 2, depth_margin = 2, joint_motion_num=10, 
            terminate_on_first = True, N_search = 100, N_loop=1000,
            display=False, dt_vis=dt_sim/40, verbose = True, print_expression=False, error_skip=0,
            **dict(N=N_fullstep, dt=dt_sim, vel_conv=1e-3, err_conv=1e-3, N_step=N_fullstep))

        schedule_dict = graph.find_schedules()
        schedule_sorted = graph.sort_schedule(schedule_dict)
        print(gtimer)
        schedule_dict = graph.find_schedules()
        schedule_sorted = graph.sort_schedule(schedule_dict)

        schedule = schedule_sorted[0]
        state_schedule = graph.idxSchedule2stateScedule(schedule, ZERO_JOINT_POSE)
        obs_names = ["box3"]

        with DynamicDetector(obs_names, aruco_map, dictionary, rs_config, T_c12, objectPose_dict['floor']) as dynamic_detector, \
                RvizPublisher(graph, obs_names) as rviz_pub:
            graph.execute_schedule_online(state_schedule, control_freq=DEFAULT_TRAJ_FREQUENCY, playback_rate=0.5,
                                          vel_conv=0.2e-2, err_conv=1e-3, T_step = 100, on_rviz=False, 
                                          obs_names=obs_names, dynamic_detector=dynamic_detector, rviz_pub=rviz_pub)
    print("===========================================")
    print("=================KILLED====================")
    print("===========================================")
t_loop = Thread(target=loop_process)
t_loop.start()

In [ ]:
loop_process.stop_now=True

In [ ]:
t_loop.is_alive()

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
ax = plt.plot(qcount_indy_list)
ax = plt.plot(qcount_panda_list)
plt.subplot(1,2,2)
ax = plt.plot(looptime_list)
print(gtimer)

# Item handling scripts

## show env image

In [ ]:
# screen_size = (1080,1920)
# kn_image_out = draw_objects(color_image, aruco_map, objectPose_dict, corner_dict, *kn_config, axis_len=0.1)
# rs_image_out = draw_objects(rs_image, aruco_map, {k:v for k,v in rs_objectPose_dict.items() if k != 'wall'}, rs_corner_dict, *rs_config, axis_len=0.1)
# kn_image_out_res = cv2.resize(kn_image_out, (rs_image_out.shape[1], rs_image_out.shape[0]))
# image_out = np.concatenate([kn_image_out_res, rs_image_out], axis=1)
# ratio = np.min(np.array(screen_size,dtype=np.float)/np.array(image_out.shape[:2],dtype=np.float))
# image_out_res = cv2.resize(image_out, (int(image_out.shape[1]*ratio), int(image_out.shape[0]*ratio)))
# cv2.imshow("test", image_out_res)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
kn_image_out = draw_objects(color_image, aruco_map, objectPose_dict, corner_dict, *kn_config, axis_len=0.1)
rs_image_out = draw_objects(rs_image, aruco_map, {k:v for k,v in rs_objectPose_dict.items() if k != 'wall'}, rs_corner_dict, *rs_config, axis_len=0.1)
plt.figure(figsize=(30,10))
plt.subplot(1,2,1)
plt.imshow(kn_image_out[:,:,[2,1,0]])
plt.subplot(1,2,2)
plt.imshow(rs_image_out[:,:,[2,1,0]])

## Register binders

In [ ]:
graph.register_binder(name='grip1', _type=Gripper2Tool, point=[0,0,0.112], link_name="panda1_hand", direction=[0,1,0])
graph.register_binder(name='grip0', _type=Gripper2Tool, point=[0,0,0.14], link_name="indy0_tcp", direction=[0,1,0])
graph.register_binder(name='floor', _type=PlacePlane, direction=[0,0,1])

## detect movable

In [ ]:
BINDER_DICT = {'goal_bd': dict(_type=PlacePlane, object_name="goal", point=[0,0,0.02], direction=[0,0,1])}
OBJECT_DICT = {'box1': dict(_type=BoxAction, hexahedral=True),
               'box2': dict(_type=BoxAction, hexahedral=True)}

In [ ]:
objectPose_dict_mv, corner_dict_mv, color_image, aruco_map_mv = detect_objects(aruco_map, dictionary)
put_point_dict = graph.register_object_gen(objectPose_dict_mv, BINDER_DICT, OBJECT_DICT, 
                 ref_tuple=(REF_NAME, objectPose_dict[REF_NAME]), link_name="base_link")

# show markers

In [ ]:
plt.figure(figsize=(10,8))
aruco_map_mv.update({'floor': aruco_map['floor']})
color_image_out = draw_objects(color_image, aruco_map_mv, objectPose_dict_mv, corner_dict_mv, *kn_config, axis_len=0.1)
plt.imshow(color_image_out[:,:,[2,1,0]])

# Register object binders

In [ ]:
register_hexahedral_binder(graph, object_name='box1', _type=PlacePlane)
register_hexahedral_binder(graph, object_name='box2', _type=PlacePlane)

# Set ui data

In [ ]:
graph.set_rviz(ZERO_JOINT_POSE)
ui_broker.set_tables()

In [ ]:
np.rad2deg(graph.planner.inp[3:-2] - graph.joints.position)

In [ ]:
graph.show_pose(graph.planner.inp[3:-2])

In [ ]:
graph.show_pose(graph.planner.inp[3:-2])

# Going back

In [ ]:
e_POS = e_POS_list[0]
self.panda.move_joint_interpolated(
    e_POS[0, graph.panda_idx], N_step=500, N_div=500)

In [ ]:
e_POS = e_POS_list[0]
if hasattr(graph, 'indy'):
    graph.indy.joint_move_to(np.rad2deg(e_POS[0,graph.indy_idx]))

In [ ]:
self.reset_panda()

In [ ]:
'x' in [u'x', u'y']

## test unavailable binder

In [ ]:
from_state = graph.snode_dict[2].state
to_state = from_state.copy()
print(to_state.node)
print(sorted(graph.binder_dict.keys()))

In [ ]:
to_state.node = (('box1', 'back_p', 'box2_right'), ('box2', 'front_p', 'grip0'))

In [ ]:
traj, new_state, error, succ = graph.test_transition(from_state, to_state, display=True, N=N_fullstep, dt=dt_sim, vel_conv=1e-3, err_conv=1e-3)

# test mesh

In [ ]:
from pkg.gjk import *

In [ ]:
Q1 = ZERO_JOINT_POSE
Q2 = ZERO_JOINT_POSE - 0.1

gtimer.reset()

gtimer.tic("svt")
swept_volume_test(Q1, Q2, graph.fixed_collision_items_list+graph.movable_collision_items_list, 
                  graph.joint_names, graph.urdf_content)
gtimer.toc("svt")

In [ ]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as mplot3d

fig = plt.figure(figsize=(15, 5))

idx_col = 205

sub = fig.add_subplot(1, 1, 1, projection="3d")
x, y, z = np.transpose(vtx2_list[idx1_list[idx_col]])
sub.plot(x, y, z, '-o')
x, y, z = np.transpose(vtx2_list[idx2_list[idx_col]])
print(GeometryItem.GLOBAL_GEO_LIST[idx1_list[idx_col]].name)
print(GeometryItem.GLOBAL_GEO_LIST[idx2_list[idx_col]].name)
sub.plot(x, y, z, '-o')
# sub.view_init(0, 90)

In [ ]:
# np.save("vtx_list", vtx_list)
# np.save("radius_list", radius_list)
# np.save("idx1_list", idx1_list)
# np.save("idx2_list", idx2_list)

# test time

In [ ]:
gtimer.reset()
found_vec = []
for _ in range(10):
    objectPose_dict_mv, corner_dict_mv, color_image, aruco_map_mv = \
        detect_objects(MOVABLE_GENERATORS, aruco_map, dictionary)
    objectPose_dict_mv.update({'floor': objectPose_dict['floor']})
    xyz_rvec_mv_dict, put_point_dict, up_point_dict = calc_put_point(
        objectPose_dict_mv, MOVABLE_GENERATORS, OBJECT_DICT, ("floor", objectPose_dict["floor"]))
    update_geometries(MOVABLE_GENERATORS.keys(), objectPose_dict_mv)
    graph.show_pose(ZERO_JOINT_POSE)
    
    dt_sim = 0.04
    T_step = 10
    N_fullstep = int(T_step / dt_sim)
    initial_state = State(tuple([(oname, put_point_dict[oname],'floor') for oname in graph.object_list]), 
                          {oname: graph.object_dict[oname].object.get_offset_tf() for oname in graph.object_list}, 
                          (graph.get_real_robot_pose() if graph.connect_indy and graph.connect_panda 
                           else ZERO_JOINT_POSE))

    goal_nodes = get_goal_nodes(initial_state, "box1", "goal_bd")
    # goal_nodes = goal_nodes[3:4]

    graph.search_graph_mp(
        initial_state = initial_state, goal_nodes = goal_nodes, swept_volume_test_jmotion=True,
        tree_margin = 2, depth_margin = 2, joint_motion_num=20, 
        terminate_on_first = True, N_search = 100, N_loop=1000,
        display=False, dt_vis=dt_sim/40, verbose = True, print_expression=False, error_skip=0,
        **dict(N=N_fullstep, dt=dt_sim, vel_conv=1e-2, err_conv=1e-3, N_step=N_fullstep))

    schedule_dict = graph.find_schedules()
    schedule_sorted = graph.sort_schedule(schedule_dict)
    found_vec.append(len(schedule_sorted)>0)
print(gtimer)

In [ ]:
print("Success rate: {} %".format(np.mean(found_vec)*100))
print(gtimer)

In [ ]:
# SVT CASE
# Success rate: 100.0 %
# show_pose: 	637.0 ms/30 = 21.22 ms (18.16/27.518)
# search_graph_mp: 	99415.0 ms/30 = 3313.844 ms (1712.408/4521.561)
# init_search: 	57.0 ms/30 = 1.908 ms (1.497/7.065)
# score_graph: 	29.0 ms/240 = 0.122 ms (0.086/0.272)
# reset_valid_node: 	14.0 ms/6030 = 0.002 ms (0.001/0.025)
# check_goal: 	4.0 ms/6177 = 0.001 ms (0.0/0.021)
# get_valid_neighbor: 	6.0 ms/540 = 0.011 ms (0.002/3.62)
# add_node_queue_leafs: 	113.0 ms/30 = 3.767 ms (2.884/5.323)
# find_schedules: 	28.0 ms/30 = 0.948 ms (0.604/1.509)
# sort_schedule: 	0.0 ms/30 = 0.01 ms (0.007/0.022)

In [ ]:
# NO SVT CASE
# Success rate: 100.0 %
# show_pose: 	609.0 ms/30 = 20.284 ms (17.837/26.163)
# search_graph_mp: 	96204.0 ms/30 = 3206.813 ms (1566.82/4633.293)
# init_search: 	53.0 ms/30 = 1.753 ms (1.495/2.953)
# score_graph: 	28.0 ms/240 = 0.116 ms (0.088/0.312)
# reset_valid_node: 	14.0 ms/6030 = 0.002 ms (0.001/0.022)
# check_goal: 	4.0 ms/6174 = 0.001 ms (0.0/0.015)
# get_valid_neighbor: 	2.0 ms/540 = 0.004 ms (0.002/0.026)
# add_node_queue_leafs: 	109.0 ms/30 = 3.65 ms (2.816/4.925)
# find_schedules: 	28.0 ms/30 = 0.941 ms (0.617/2.113)
# sort_schedule: 	0.0 ms/30 = 0.01 ms (0.007/0.015)